In [ ]:
!python version;

python3: can't open file 'version': [Errno 2] No such file or directory


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
!pip install contractions
# import contractions
import re
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df= pd.read_csv("/content/drive/MyDrive/NLP/ANLP_HW1_Dataset.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,7,8,9,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
####ONLY KEEPING'REVIEW BODY' AND 'STAR_RATING' AS DEFINED IN THE PROBLEM STATEMENT######################
dataset = df[['star_rating','review_body']]

In [ ]:
######################################DISPLAYING STATISTICS OF ALL THREE CLASSES##################################
Positive_Reviews = dataset[dataset.star_rating ==4].shape[0] + dataset[dataset.star_rating ==5].shape[0]
Negative_Reviews =  dataset[dataset.star_rating ==1].shape[0] + dataset[dataset.star_rating ==2].shape[0]
Neutral_Reviews = dataset[dataset.star_rating ==3].shape[0]
print('Statistics- Positive Reviews:', Positive_Reviews , ', Negative Reviews:', Negative_Reviews, ', Neutral_Reviews:', Neutral_Reviews)

Statistics- Positive Reviews: 733804 , Negative Reviews: 107682 , Neutral_Reviews: 60259


In [ ]:
##################DISCARDING THE REVIEWS WHICH HAVE 'STAR_RATING' AS 3 I.E NEUTRAL REVIEWS###################
dataset = dataset[dataset.star_rating != 3]

In [ ]:
#####################ASSIGNING LABELS TO THE DATA; 1 CORRESPONDS TO RATINGS {4,5}, 0 CORRESPONDS TO RATINGS {1,2}#############
dataset['star_rating'] = pd.to_numeric(dataset['star_rating'],errors='coerce')
dataset['labels'] = np.where(dataset['star_rating']>=4, 1, 0)

In [ ]:
#######################DIVIDING POSITIVE & NEGATIVE REVIEWS AND SELECTING A RANDOM SAMPLE OF 10000 DATAROWS###############################
dataset_pos= dataset.loc[dataset['labels'] == 1]
dataset_pos= dataset.sample(100000)
dataset_neg= dataset.loc[dataset['labels'] == 0]
dataset_neg = dataset.sample(100000)
dataset= dataset_pos
dataset= dataset.append(dataset_neg)
dataset = dataset.sample(frac = 1)

In [ ]:
####################DISPLAYING AVERAGE LENGTH OF REVIEWS BEFORE DATA CLEANING###################
Before_DataCleaning = dataset['review_body'].astype(str).apply(lambda x : len(str(x))).mean()
print('Average Length of Reviews before Data Cleaning',Before_DataCleaning)

Average Length of Reviews before Data Cleaning 187.05533


In [ ]:
#######################CONVERT ALL REVIEWS INTO LOWER CASE###########
dataset['review_body']= dataset['review_body'].apply(lambda txt: str(txt).lower())

In [ ]:
##########################REMOVE HTML and URLs FROM CUSTOMER REVIEWS#################
###########1. REMOVING URL's####################
dataset['review_body']= dataset['review_body'].apply(lambda txt: re.sub(r"http\S+", "", txt) )
##########2. REMOVING HTML####################
dataset['review_body']= dataset['review_body'].apply(lambda txt: BeautifulSoup(txt, 'lxml').get_text())

In [ ]:
##########################REMOVE NON-ALPHABETICAL CHARACTERS#######################
dataset['review_body']= dataset['review_body'].apply(lambda txt: re.sub("[^a-zA-Z]+", " ",txt) ) 

In [ ]:
#########################REMOVE THE EXTRA SPACES BETWEEN WORDS###################
dataset['review_body']= dataset['review_body'].apply(lambda txt: re.sub(" +"," ", txt))

In [ ]:
########################PERFORM CONTRACTIONS#######################################
import contractions
dataset['review_body']= dataset['review_body'].apply(lambda txt: contractions.fix(txt))

In [ ]:
######################DISPLAYING AVERAGE LENGTH OF REVIEWS AFTER DATA CLEANING & BEFORE PRE PROCESSING#############
AFter_DataCleaning = dataset['review_body'].apply(len).mean()
print('Average Length of Reviews after Data Cleaning',AFter_DataCleaning)
print('Average Length of Reviews before Pre-Processing',AFter_DataCleaning)

Average Length of Reviews after Data Cleaning 179.510425
Average Length of Reviews before Pre-Processing 179.510425


In [ ]:
#####################REMOVE STOPWORDS##########################################
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def Remove_Stopwords(message):
  tokens = message.split(" ")
  stop_words_list = set (stopwords.words('english'))
  clean_message = [word for word in tokens if not word in stop_words_list]
  return [(" ").join(clean_message)]
  
dataset['review_body']= dataset['review_body'].apply(lambda txt: Remove_Stopwords(txt))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
######################################PERFORM LEMMATIZATION#############################
from nltk.stem import WordNetLemmatizer
def lemmatize(message):
    word_lemma = WordNetLemmatizer()
    lemmatize_word= [word_lemma.lemmatize(word) for word in message]
    return (" ").join(lemmatize_word)
dataset['review_body']= dataset['review_body'].apply(lambda txt: lemmatize(txt))

In [ ]:
######################DISPLAYING AVERAGE LENGTH OF REVIEWS AFTER PRE PROCESSING#############
AFter_Preprocessing = dataset['review_body'].apply(len).mean()
print('Average Length of Reviews after Pre-Processing',AFter_Preprocessing)

Average Length of Reviews after Pre-Processing 113.981


In [ ]:
############################################FEATURE EXTRACTION & SPLITTING DATA FOR TRAINING & TESTING######################
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

data_train, data_test, y_train, y_test = train_test_split(dataset['review_body'], dataset['labels'], test_size = 0.2)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
Training_data = tfidf.fit_transform(data_train)
Testing_data = tfidf.transform(data_test)

In [ ]:
######################################PERCEPTRON MODEL##################################
from sklearn.linear_model import Perceptron
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
Perceptron_Model= Perceptron()
Perceptron_Model.fit(Training_data,y_train)
Prediction = Perceptron_Model.predict(Testing_data)
print('Perceptron Model(Testing_data)--Accuracy:',accuracy_score(y_test,Prediction))
print('Perceptron Model(Testing_data)--Precision Score:',precision_score(y_test,Prediction))
print('Perceptron Model(Testing_data)--Recall Score:',recall_score(y_test,Prediction))
print('Perceptron Model(Testing_data)--F1 Score:',f1_score(y_test,Prediction))
Prediction = Perceptron_Model.predict(Training_data)
print('Perceptron Model(Training_data)--Accuracy:',accuracy_score(y_train,Prediction))
print('Perceptron Model(Training_data)--Precision Score:',precision_score(y_train,Prediction))
print('Perceptron Model(Training_data)--Recall Score:',recall_score(y_train,Prediction))
print('Perceptron Model(Training_data)--F1 Score:',f1_score(y_train,Prediction))

Perceptron Model(Testing_data)--Accuracy: 0.919625
Perceptron Model(Testing_data)--Precision Score: 0.9516885958598451
Perceptron Model(Testing_data)--Recall Score: 0.9555677613320999
Perceptron Model(Testing_data)--F1 Score: 0.9536242336819329
Perceptron Model(Training_data)--Accuracy: 0.9480125
Perceptron Model(Training_data)--Precision Score: 0.9686538225902942
Perceptron Model(Training_data)--Recall Score: 0.9713141709007596
Perceptron Model(Training_data)--F1 Score: 0.9699821726295733


In [ ]:
############################################SUPPORT VECTOR MACHINE#######################
from sklearn.svm import SVC
svm_Model = SVC(kernel='linear',max_iter=500)
svm_Model.fit(Training_data,y_train)
prediction = svm_Model.predict(Testing_data)
print('SVM Model(Testing_data)--Accuracy:',accuracy_score(y_test,prediction))
print('SVM Model(Testing_data)--Precision Score:',precision_score(y_test,prediction))
print('SVM Model(Testing_data)--Recall Score:',recall_score(y_test,prediction))
print('SVM Model(Testing_data)--F1 Score:',f1_score(y_test,prediction))
prediction = svm_Model.predict(Training_data)
print('SVM Model(Training_data)--Accuracy:',accuracy_score(y_train,prediction))
print('SVM Model(Training_data)--Precision Score:',precision_score(y_train,prediction))
print('SVM Model(Training_data)--Recall Score:',recall_score(y_train,prediction))
print('SVM Model(Training_data)--F1 Score:',f1_score(y_train,prediction))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM Model(Testing_data)--Accuracy: 0.850925
SVM Model(Testing_data)--Precision Score: 0.900708227192565
SVM Model(Testing_data)--Recall Score: 0.9301572617946346
SVM Model(Testing_data)--F1 Score: 0.9151959041456303
SVM Model(Training_data)--Accuracy: 0.8515875
SVM Model(Training_data)--Precision Score: 0.9015295362345243
SVM Model(Training_data)--Recall Score: 0.9299513591257652
SVM Model(Training_data)--F1 Score: 0.915519915754721


In [ ]:
#########################################LOGISTIC REGRESSION##############################
from sklearn.linear_model import LogisticRegression
logreg_Model = LogisticRegression(max_iter = 500)
logreg_Model.fit(Training_data, y_train)
prediction = logreg_Model.predict(Testing_data)
print('Logistic_Regression(Testing_data) Model--Accuracy:',accuracy_score(y_test,prediction))
print('Logistic_Regression(Testing_data) Model--Precision Score:',precision_score(y_test,prediction))
print('Logistic_Regression(Testing_data) Model--Recall Score:',recall_score(y_test,prediction))
print('Logistic_Regression(Testing_data) Model--F1 Score:',f1_score(y_test,prediction))
prediction = logreg_Model.predict(Training_data)
print('Logistic_Regression(Training_data) Model--Accuracy:',accuracy_score(y_train,prediction))
print('Logistic_Regression(Training_data) Model--Precision Score:',precision_score(y_train,prediction))
print('Logistic_Regression(Training_data) Model--Recall Score:',recall_score(y_train,prediction))
print('Logistic_Regression(Training_data) Model--F1 Score:',f1_score(y_train,prediction))

Logistic_Regression(Testing_data) Model--Accuracy: 0.9397
Logistic_Regression(Testing_data) Model--Precision Score: 0.9481394830659536
Logistic_Regression(Testing_data) Model--Recall Score: 0.9841003700277521
Logistic_Regression(Testing_data) Model--F1 Score: 0.9657852927825692
Logistic_Regression(Training_data) Model--Accuracy: 0.94645
Logistic_Regression(Training_data) Model--Precision Score: 0.9527736498036015
Logistic_Regression(Training_data) Model--Recall Score: 0.9869977811666583
Logistic_Regression(Training_data) Model--F1 Score: 0.9695838007469151


In [ ]:
######################################MULTINOMINAL REGRESSION#############################
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
MulNb_Model = MultinomialNB(alpha=6.5, fit_prior=False)
MulNb_Model.fit(Training_data,y_train)
prediction = MulNb_Model.predict(Testing_data)
print('Multinominal_Naive_Bayes(Testing_data) Model--Accuracy:',accuracy_score(y_test,prediction))
print('Multinominal_Naive_Bayes(Testing_data) Model--Precision Score:',precision_score(y_test,prediction))
print('Multinominal_Naive_Bayes(Testing_data) Model--Recall Score:',recall_score(y_test,prediction))
print('Multinominal_Naive_Bayes(Testing_data) Model--F1 Score:',f1_score(y_test,prediction))

prediction = MulNb_Model.predict(Training_data)
print('Multinominal_Naive_Bayes(Training_data) Model--Accuracy:',accuracy_score(y_train,prediction))
print('Multinominal_Naive_Bayes(Training_data) Model--Precision Score:',precision_score(y_train,prediction))
print('Multinominal_Naive_Bayes(Training_data) Model--Recall Score:',recall_score(y_train,prediction))
print('Multinominal_Naive_Bayes(Training_data) Model--F1 Score:',f1_score(y_train,prediction))

Multinominal_Naive_Bayes(Testing_data) Model--Accuracy: 0.8843
Multinominal_Naive_Bayes(Testing_data) Model--Precision Score: 0.887351983039454
Multinominal_Naive_Bayes(Testing_data) Model--Recall Score: 0.9921658186864015
Multinominal_Naive_Bayes(Testing_data) Model--F1 Score: 0.9368363586733999
Multinominal_Naive_Bayes(Training_data) Model--Accuracy: 0.886825
Multinominal_Naive_Bayes(Training_data) Model--Precision Score: 0.8899077214393639
Multinominal_Naive_Bayes(Training_data) Model--Recall Score: 0.9918257312392943
Multinominal_Naive_Bayes(Training_data) Model--F1 Score: 0.9381066965628503
